In [1]:
import duckdb

In [2]:
con = duckdb.connect()

con.sql("INSTALL duckpgq FROM community;")

con.load_extension("duckpgq")

In [3]:
con.sql("""
CREATE TABLE Person as select * from 'https://gist.githubusercontent.com/Dtenwolde/2b02aebbed3c9638a06fda8ee0088a36/raw/8c4dc551f7344b12eaff2d1438c9da08649d00ec/person-sf0.003.csv';
CREATE TABLE Person_knows_person as select * from 'https://gist.githubusercontent.com/Dtenwolde/81c32c9002d4059c2c3073dbca155275/raw/8b440e810a48dcaa08c07086e493ec0e2ec6b3cb/person_knows_person-sf0.003.csv';

CREATE PROPERTY GRAPH snb
  VERTEX TABLES (
    Person
  )
EDGE TABLES (
  Person_knows_person 	SOURCE KEY (Person1Id) REFERENCES Person (id)
                        DESTINATION KEY (Person2Id) REFERENCES Person (id)
  LABEL Knows
);
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [4]:
con.sql("""
FROM GRAPH_TABLE (snb
  MATCH (a:Person)-[k:knows]->(b:Person)
  COLUMNS (a.id, b.id)
)
LIMIT 1;
""")

┌───────┬────────────────┐
│  id   │      id_1      │
│ int64 │     int64      │
├───────┼────────────────┤
│    14 │ 10995116277782 │
└───────┴────────────────┘

In [5]:
con.sql("""
FROM GRAPH_TABLE (snb 
  MATCH p = ANY SHORTEST (a:person)-[k:knows]->{1,3}(b:Person) 
  COLUMNS (a.id, b.id, path_length(p))
) 
LIMIT 1;
""")

┌───────┬────────────────┬────────────────┐
│  id   │      id_1      │ path_length(p) │
│ int64 │     int64      │     int64      │
├───────┼────────────────┼────────────────┤
│    14 │ 10995116277782 │              1 │
└───────┴────────────────┴────────────────┘

In [6]:
con.sql("""
FROM local_clustering_coefficient(snb, person, knows);
""")

┌────────────────┬──────────────────────────────┐
│       id       │ local_clustering_coefficient │
│     int64      │            float             │
├────────────────┼──────────────────────────────┤
│             14 │                   0.33333334 │
│             16 │                          0.5 │
│             32 │                    0.8333333 │
│  2199023255557 │                    0.6666667 │
│  2199023255573 │                          1.0 │
│  2199023255594 │                    0.1904762 │
│  4398046511139 │                          0.0 │
│  6597069766702 │                          0.0 │
│  8796093022234 │                          0.0 │
│  8796093022237 │                    0.6666667 │
│        ·       │                           ·  │
│        ·       │                           ·  │
│        ·       │                           ·  │
│ 30786325577740 │                          1.0 │
│ 32985348833291 │                          0.0 │
│ 32985348833318 │                          0.0 │


In [7]:
con.sql("""
SELECT lastName, speaks FROM Person LIMIT 5
""")

┌────────────┬──────────┐
│  lastName  │  speaks  │
│  varchar   │ varchar  │
├────────────┼──────────┤
│ Forouhar   │ fa;ku;en │
│ Zakrzewski │ pl;en    │
│ Gonzalez   │ es;en    │
│ Mettacara  │ my;en    │
│ Ali        │ ur;en    │
└────────────┴──────────┘

In [8]:
con.sql("""
SELECT * FROM Person_knows_person LIMIT 5
""")

┌─────────────────────────┬───────────┬────────────────┐
│      creationDate       │ Person1Id │   Person2Id    │
│        timestamp        │   int64   │     int64      │
├─────────────────────────┼───────────┼────────────────┤
│ 2012-10-07 02:24:40.381 │        14 │ 10995116277782 │
│ 2012-07-08 15:27:12.264 │        14 │ 24189255811081 │
│ 2012-11-26 06:45:21.004 │        14 │ 26388279066668 │
│ 2011-11-08 06:05:10.543 │        16 │  2199023255594 │
│ 2012-06-14 12:43:25.817 │        16 │ 26388279066655 │
└─────────────────────────┴───────────┴────────────────┘